In [ ]:
### Bloque 1: Importación de librerías

En este bloque se importan las librerías necesarias para el desarrollo del notebook:

- `numpy` se utiliza para realizar operaciones matemáticas y trigonométricas (por ejemplo, conversiones a radianes, raíces cuadradas, etc.).
- `matplotlib.pyplot` se usa para graficar el robot SCARA en el plano XY y visualizar cada configuración articular.
- `imageio.v2` permite crear un archivo GIF a partir de una secuencia de imágenes estáticas.
- `os` se emplea para manejar rutas y crear la carpeta `media/` donde se guardan los frames y el GIF.

In [63]:
# Celda 1: imports básicos

import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import os


In [ ]:
### Bloque 2: Parámetros geométricos y cinemática directa

En este bloque se definen los parámetros geométricos del robot SCARA real y la función de **cinemática directa**:

- `L1` y `L2` representan las longitudes de los dos eslabones horizontales del robot (en milímetros).
- `H_MIN` es la altura mínima del efector final respecto a la base, a partir de la cual se mide el desplazamiento vertical `d3`.
- `D3_MIN` y `D3_MAX` definen el rango admisible de movimiento de la articulación prismática (eje Z).

La función `fk_scara(q1_deg, q2_deg, d3_mm)` recibe como entrada los ángulos articulares `q1` y `q2` en **grados**, además del desplazamiento vertical `d3` en **milímetros**, y devuelve la posición cartesiana del efector:

- Convierte `q1` y `q2` a radianes para poder usar funciones trigonométricas.
- Calcula las coordenadas `x` y `y` en el plano usando las ecuaciones clásicas de un manipulador plano de dos eslabones.
- Calcula `z` como la suma de la altura mínima `H_MIN` más el desplazamiento vertical `d3`.

Esta función se utiliza más adelante para **verificar** que la cinemática inversa está funcionando correctamente.


In [31]:
# Celda 2: parámetros geométricos y cinemática directa

# Parámetros del robot (mm)
L1 = 80.0      # longitud eslabón 1
L2 = 80.0      # longitud eslabón 2
H_MIN = 23.45  # altura mínima del efector

D3_MIN = 0.0       # recorrido mínimo en Z (mm)
D3_MAX = 30.85     # recorrido máximo en Z (mm)

def fk_scara(q1_deg, q2_deg, d3_mm):
    """
    Cinemática directa del SCARA RRP.
    Entradas:
        q1_deg, q2_deg : ángulos de las articulaciones 1 y 2 (grados)
        d3_mm          : desplazamiento vertical desde H_MIN (mm)
    Salidas:
        x_mm, y_mm, z_mm (mm)
    """
    q1 = np.deg2rad(q1_deg)
    q2 = np.deg2rad(q2_deg)

    x = L1 * np.cos(q1) + L2 * np.cos(q1 + q2)
    y = L1 * np.sin(q1) + L2 * np.sin(q1 + q2)
    z = H_MIN + d3_mm

    return x, y, z


In [ ]:
### Bloque 3: Función de cinemática inversa del SCARA

En este bloque se implementa la función `ik_scara(x_mm, y_mm, z_mm, elbow="down")`, que resuelve la **cinemática inversa**:

- La entrada es un punto cartesiano deseado `(x, y, z)` en milímetros y un parámetro `elbow` que indica si se desea la configuración de **codo abajo** (`"down"`) o **codo arriba** (`"up"`).
- Primero se calcula la distancia radial `r` desde el origen hasta el punto `(x, y)` para saber si el objetivo está dentro del espacio de trabajo.
- Se aplica la **ley de cosenos** para obtener el ángulo `q2`, asegurando que el valor de `cos_q2` esté entre -1 y 1 para evitar errores numéricos.
- Dependiendo del parámetro `elbow`, se elige el signo de `sin_q2`, lo que define la postura del codo.
- Con `q2` ya calculado, se obtiene `q1` usando relaciones geométricas (`k1`, `k2`) y la función `arctan2`.
- Para el eje vertical se calcula `d3` como `z - H_MIN` y luego se satura al rango físico `[D3_MIN, D3_MAX]`.
- Finalmente, `q1` y `q2` se convierten a grados y se regresan junto con `d3`.

Si el punto no es alcanzable en XY, la función lanza un `ValueError` indicando que el objetivo está fuera del espacio de trabajo.


In [33]:
# Celda 3: cinemática inversa del SCARA RRP

def ik_scara(x_mm, y_mm, z_mm, elbow="down"):
    """
    Cinemática inversa de un SCARA RRP.

    Entradas:
        x_mm, y_mm, z_mm : posición objetivo del efector (mm)
        elbow : "down" o "up" para elegir configuración de codo

    Salidas:
        q1_deg, q2_deg, d3_mm
    """
    # Distancia radial en el plano XY
    r2 = x_mm**2 + y_mm**2
    r = np.sqrt(r2)

    # Comprobación de alcanzabilidad en XY (círculo de trabajo)
    if r > (L1 + L2) or r < abs(L1 - L2):
        raise ValueError("Punto fuera del espacio de trabajo en XY")

    # q2 por ley de cosenos
    cos_q2 = (r2 - L1**2 - L2**2) / (2 * L1 * L2)
    cos_q2 = np.clip(cos_q2, -1.0, 1.0)  # por seguridad numérica

    if elbow == "down":
        sin_q2 = -np.sqrt(1 - cos_q2**2)
    else:
        sin_q2 =  np.sqrt(1 - cos_q2**2)

    q2 = np.arctan2(sin_q2, cos_q2)

    # q1
    k1 = L1 + L2 * cos_q2
    k2 = L2 * sin_q2
    q1 = np.arctan2(y_mm, x_mm) - np.arctan2(k2, k1)

    # Eje Z
    d3_mm = z_mm - H_MIN
    d3_mm = max(D3_MIN, min(D3_MAX, d3_mm))  # saturamos al rango físico

    # Convertimos a grados
    q1_deg = np.rad2deg(q1)
    q2_deg = np.rad2deg(q2)

    return q1_deg, q2_deg, d3_mm


In [ ]:
### Bloque 4: Definición de un punto cartesiano de prueba

En este bloque se fija manualmente un punto objetivo en el espacio cartesiano:

- `x_obj`, `y_obj` y `z_obj` representan las coordenadas deseadas del efector final en milímetros.

Estos valores se utilizan como caso de prueba individual para:
1. Calcular la cinemática inversa (`q1`, `q2`, `d3`) que lleva el efector a esa posición.
2. Verificar, mediante cinemática directa, que el modelo matemático realmente devuelve el mismo punto.

Este enfoque de **asignación directa** facilita repetir el procedimiento con distintos puntos para llenar la tabla de pruebas del reporte.


In [35]:
# Celda 4: definir un punto cartesiano objetivo (ejemplo)

x_obj = 140.0   # [mm]
y_obj = 40.0    # [mm]
z_obj = 35.0    # [mm]

print(f"Punto deseado -> x = {x_obj} mm, y = {y_obj} mm, z = {z_obj} mm")


Punto deseado -> x = 140.0 mm, y = 40.0 mm, z = 35.0 mm


In [ ]:
### Bloque 5: Cálculo de la solución de cinemática inversa y verificación

En este bloque se llama a la función de cinemática inversa para obtener las variables articulares correspondientes al punto objetivo:

- Se envuelven las llamadas a `ik_scara` dentro de un bloque `try/except` para capturar el posible `ValueError` cuando el punto no es alcanzable.
- Si el cómputo tiene éxito, se muestran en pantalla:
  - Los valores de `q1`, `q2` en grados.
  - El valor de `d3` en milímetros.
- A continuación, se llama a la función `fk_scara` con esos mismos valores para obtener `x_fk`, `y_fk`, `z_fk`.
- La finalidad de esta verificación es comprobar que:
  \[
  (x_{\text{fk}}, y_{\text{fk}}, z_{\text{fk}}) \approx (x_{\text{obj}}, y_{\text{obj}}, z_{\text{obj}})
  \]
  lo que indica que la implementación de la cinemática inversa y directa es coherente.

Si el punto está fuera del espacio de trabajo, solo se imprime el mensaje de error sin provocar que el notebook se detenga.


In [37]:
# Celda 5: resolver cinemática inversa y verificar con cinemática directa

try:
    q1_sol, q2_sol, d3_sol = ik_scara(x_obj, y_obj, z_obj, elbow="down")

    print("Solución de cinemática inversa:")
    print(f"q1 = {q1_sol:.2f} °")
    print(f"q2 = {q2_sol:.2f} °")
    print(f"d3 = {d3_sol:.2f} mm")

    # Verificación
    x_fk, y_fk, z_fk = fk_scara(q1_sol, q2_sol, d3_sol)
    print("\nVerificación por cinemática directa:")
    print(f"x_fk = {x_fk:.2f} mm")
    print(f"y_fk = {y_fk:.2f} mm")
    print(f"z_fk = {z_fk:.2f} mm")

except ValueError as e:
    print("Error:", e)


Solución de cinemática inversa:
q1 = 40.44 °
q2 = -48.99 °
d3 = 11.55 mm

Verificación por cinemática directa:
x_fk = 140.00 mm
y_fk = 40.00 mm
z_fk = 35.00 mm


In [ ]:
### Bloque 6: Definición de la trayectoria cartesiana

En este bloque se define una lista de puntos cartesianos `puntos_cartesianos` que constituirán una trayectoria para el robot:

- Cada elemento de la lista es una tupla `(x, y, z)` que representa una posición objetivo del efector.
- Se escogen varios puntos dentro del espacio de trabajo para que el movimiento del robot sea variado (por ejemplo, adelante, arriba, atrás, etc.).
- Se imprime la lista numerada para poder identificar fácilmente cada punto en las salidas posteriores y en el reporte.

Estos puntos se usarán en el bloque siguiente para generar un GIF donde el SCARA visita cada uno en secuencia usando cinemática inversa.


In [39]:
# Celda 6: definir varios puntos cartesianos para la trayectoria (ejemplo)

puntos_cartesianos = [
    (160.0,   0.0,  25.0),  # punto 1
    (140.0,  40.0,  30.0),  # punto 2
    (110.0, -60.0,  35.0),  # punto 3
    (130.0,   0.0,  40.0),  # punto 4
]

print("Puntos cartesianos de la trayectoria:")
for i, (x, y, z) in enumerate(puntos_cartesianos, start=1):
    print(f"P{i}: x={x} mm, y={y} mm, z={z} mm")


Puntos cartesianos de la trayectoria:
P1: x=160.0 mm, y=0.0 mm, z=25.0 mm
P2: x=140.0 mm, y=40.0 mm, z=30.0 mm
P3: x=110.0 mm, y=-60.0 mm, z=35.0 mm
P4: x=130.0 mm, y=0.0 mm, z=40.0 mm


In [ ]:
### Bloque 7: Generación del GIF de la trayectoria con cinemática inversa

Este bloque recorre todos los puntos definidos en `puntos_cartesianos` y genera un GIF de la trayectoria:

1. Se crea la carpeta `media/` (si no existe) para almacenar las imágenes temporales y el GIF final.
2. Para cada punto `(x_obj, y_obj, z_obj)`:
   - Se resuelve la cinemática inversa `ik_scara` para obtener `q1`, `q2` y `d3`.
   - Se verifica con `fk_scara` que la posición alcanzada \((x_{\text{fk}}, y_{\text{fk}}, z_{\text{fk}})\) corresponde al objetivo.
   - Se calculan las posiciones intermedias de las articulaciones (origen, unión entre eslabones y efector) para poder dibujar el robot en 2D.
   - Se genera una figura con `matplotlib` que muestra el robot en el plano XY y se guarda como imagen PNG en la carpeta `media/`.
   - Cada imagen se carga y se agrega a la lista `frames`.

3. Una vez generadas todas las imágenes válidas, se crea el GIF:

```python
gif_path = "media/scara_ik_trayectoria.gif"
imageio.mimsave(gif_path, frames, duration=0.8, loop=0)


In [57]:
# Celda 7: generar GIF de la trayectoria usando cinemática inversa

os.makedirs("media", exist_ok=True)

frames = []

for i, (x_obj, y_obj, z_obj) in enumerate(puntos_cartesianos, start=1):
    print(f"\nPunto {i}: x={x_obj} mm, y={y_obj} mm, z={z_obj} mm")

    try:
        # 1) Resolver IK
        q1_deg, q2_deg, d3_mm = ik_scara(x_obj, y_obj, z_obj, elbow="down")
        print(f"  IK -> q1={q1_deg:.2f}°, q2={q2_deg:.2f}°, d3={d3_mm:.2f} mm")

        # 2) Verificar con FK
        x_fk, y_fk, z_fk = fk_scara(q1_deg, q2_deg, d3_mm)
        print(f"  FK -> x={x_fk:.2f} mm, y={y_fk:.2f} mm, z={z_fk:.2f} mm")

        # 3) Calcular posiciones de articulaciones para dibujar
        q1_rad = np.deg2rad(q1_deg)
        q2_rad = np.deg2rad(q2_deg)

        x0, y0 = 0.0, 0.0
        x1 = L1 * np.cos(q1_rad)
        y1 = L1 * np.sin(q1_rad)
        x2 = x1 + L2 * np.cos(q1_rad + q2_rad)
        y2 = y1 + L2 * np.sin(q1_rad + q2_rad)

        # 4) Graficar el SCARA
        fig, ax = plt.subplots()
        ax.plot([x0, x1, x2], [y0, y1, y2], marker="o")
        ax.set_xlabel("X [mm]")
        ax.set_ylabel("Y [mm]")
        ax.set_title(f"Punto {i} - q1={q1_deg:.1f}°, q2={q2_deg:.1f}°, d3={d3_mm:.1f} mm")
        ax.set_xlim(-200, 200)
        ax.set_ylim(-200, 200)
        ax.set_aspect("equal", "box")
        ax.grid(True)

        # 5) Guardar frame
        filename = f"media/frame_ik_{i:03d}.png"
        fig.savefig(filename)
        plt.close(fig)

        frames.append(imageio.imread(filename))

    except ValueError as e:
        print("  Punto fuera del espacio de trabajo, se omite en el GIF.")
        continue

gif_path = "media/scara_ik_trayectoria.gif"
if frames:
    # loop=0  -> bucle infinito en la mayoría de visores
    imageio.mimsave(gif_path, frames, duration=1000, loop=0)
    print("\nGIF generado en:", gif_path)
else:
    print("\nNo se generaron frames; revisa los puntos ingresados.")



Punto 1: x=160.0 mm, y=0.0 mm, z=25.0 mm
  IK -> q1=0.00°, q2=-0.00°, d3=1.55 mm
  FK -> x=160.00 mm, y=0.00 mm, z=25.00 mm

Punto 2: x=140.0 mm, y=40.0 mm, z=30.0 mm
  IK -> q1=40.44°, q2=-48.99°, d3=6.55 mm
  FK -> x=140.00 mm, y=40.00 mm, z=30.00 mm

Punto 3: x=110.0 mm, y=-60.0 mm, z=35.0 mm
  IK -> q1=9.84°, q2=-76.91°, d3=11.55 mm
  FK -> x=110.00 mm, y=-60.00 mm, z=35.00 mm

Punto 4: x=130.0 mm, y=0.0 mm, z=40.0 mm
  IK -> q1=35.66°, q2=-71.32°, d3=16.55 mm
  FK -> x=130.00 mm, y=0.00 mm, z=40.00 mm

GIF generado en: media/scara_ik_trayectoria.gif
